## Import libraries

In [ ]:
# import libraries and modules
import chromdyn
from chromdyn import (
    ChromatinDynamics,
    TopologyGenerator,
    HiCManager, save_pdb,
    PlatformManager,
    Analyzer,
    TrajectoryLoader,
)
import pandas as pd
print('Loaded chromdyn version: ' + chromdyn.__version__ )
# to inspect the location of chromdyn on the filesystem, uncomment the lines below
# import inspect, os
# print('chromdyn located at:', os.path.dirname(inspect.getfile(chromdyn)))

## Step 1: Generate topology
A topology object needs to be created that holds the monomer identities and their connectivity, which is passed to the simulation

- Initialize topology generator object: `topo = TopologyGenerator()`

- Generate topology: `topo.gen_top(chain_lens=[N, M], chain_names=["chr1", "chr2"], isRing=[0,1], types='A')`

    - Default topology for a chain of size N: `topo.gen_top([N])` or `topo.gen_top([N, M])` for topology of two chains of sizes N and M

    - Specifying chain names: `chain_names=["chr1", "chr2"])`

    - Specifying ring topologies, the above code makes `chr1` a linear and `chr2` a circular chain

    - Specifying monomer types
        - string: `types=A`, all monomer types are labeled by the string
        - list: `types=['A']*n + ['B']*m`, the first `n` monomers are labeled `A` and next `m` monomers are labeled `B`--ensure `N+M=n+m`
        - keyword: `types='unique'`, all monomers are uniquely labeled as `M1, M2, M3 ...`
        - file: `types=sequence.txt`, the first coloumn should be the monomer index and second column the label--all monomers are labeled accrdingly
    
- The topology object can be accessed via `topo.topology` and the sequence of types is stored in `topo.atom_types`
- The topology can be printed on the console using `topo.print_top` and saved in a file using `topo.save_top('filename.txt')`

## Step 2: Initialize simulation object
The simulator object `ChromatinDynamics` needs to be initialized with 
- `topology` (required)
- `name: str` = "ChromatinDynamics" (this name will be used for log files)
- `platform_name: str` = "CUDA" (check available platforms using `PlatformManager` or by running on the terminal: `python -m chromdyn.check_install`)
- `output_dir: str` = "output" (if the directory does not exist it will be created)
- `console_stream: bool` = True (choose to output on the terminal)
- `write_logs: bool` = True (whether log files will be created, output_dir is created only when this is True)

## Step 3: Add forces
Forces are handled by `force_field_manager`-- typical forces that can be added are harmonic bonds, soft-core self avoidance, confinements, COM remover, etc.

## Step 4: Setup simulations
This is where the simulation context is generated. 
- The setup can take the following arguments:
    - init_struct: Union[str, NDArray] = "randomwalk" (one may also specify "random"--a random gas; 'saw3d', and a (N,3) np.array specifying positions )
    - integrator: str = "langevin" (other options are "brownian", "active-langevin", and "active-brownian")
    - temperature: float = 120.0
    - timestep: float = 0.01 
    - friction: float = 0.1 
    - save_pos: bool = True (whether positions are saved into a file or not, if True needs write_logs=True)
    - save_energy: bool = True  (whether energies are saved into a file or not, if True needs write_logs=True)
    - stability_check: bool = True (whether to check for stability and reinitialize the velocites when the KE or PE explode)
    - stability_report_interval: int = 500 
    - energy_report_interval: int = 1000
    - pos_report_interval: int = 1000

- Upon setting up the output (console or log) shows the energies in the system from the initial structure
- The files corresponding to positions (_positions.cndb) and energies (_energy_report.txt) are generated and the terminal output gives the filenames
- The stability file will be generated and will be populated only if stability issues arise and velocity reinitialization is implemented

## Step 5: Run simulation and save data
- The simulation after setup can be run for any number of integer steps using `sim.run(n_steps, report=True)`
- The `report` flag is used to detrmine if the positions and/or energies from this run are saved or not, e.g., for equiliibration runs one should set it to `False`
- At any point a summary of energies/forces in the system can be accessed via `sim.print_force_info()`
- Additionally, a snapshot maybe saved as a pdb for visualization using `save_pdb(sim)`; the filename is autogenerated from the step number
- Finally, the reports must be saved using `sim.save_reports()`; ofcourse, this is not required if `write_logs=False`

## Step 6: Analyze simulated trajectory
- The simulated trajectory can be loaded from the positions.cndb file using `TrajectoryLoader`
- analyses like radius of gyration can be computed
- Hi-C contact maps can be generated using `HiCManager`


In [2]:
# Step 1: topology
topo = TopologyGenerator()
topo.gen_top([100, 50])

In [ ]:
# Step 2a: get platforms 
pm = PlatformManager()
pm.list_platforms()

In [ ]:
# Step 2b: initialize simulation
sim = ChromatinDynamics(topo.topology, name = 'test', platform_name = "CPU", output_dir = "output", console_stream = True)

# Step 3: add forces
sim.force_field_manager.add_harmonic_bonds()
sim.force_field_manager.add_self_avoidance()
sim.force_field_manager.add_flat_bottom_harmonic()


In [ ]:
# Step 4: set up simulation
sim.simulation_setup(
            init_struct='randomwalk',
            integrator='langevin',
            temperature=120.0,
            timestep=0.01,
            save_pos=True,
            save_energy=True,              
            )

In [ ]:
# Step 5: run simulation

sim.run(5_000, report=False) # equilibration
sim.print_force_info() # check forces
sim.run(20_000, report=True) # production
save_pdb(sim) # save pdbs of individual configs
sim.print_force_info()
sim.save_reports() # Simulations done -- save reports

In [ ]:
# Step 6a: Analysis
# you may also specify directly the position file
traj_file = sim.reporters.get('position').filename
print(f'Trajectory file: { traj_file }')
# position analysis
xyz = TrajectoryLoader.load(traj_file)
print(f'Loaded trajectory with shape: { xyz.shape }')
print(f'Radius of gyration: { Analyzer.compute_RG(xyz).mean() }')


In [ ]:
# install matplotlib if you don't have it: pip install matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

energy_file = sim.reporters.get('energy').filename
print(f'Trajectory file: { traj_file } | Energy file: { energy_file }')
energies = pd.read_csv(energy_file,sep=r"\s+", engine="python")
# energy components
energy_comps= energies.keys().tolist()[5:]  # exclude time column
for comp in energy_comps:
    mean_energy = energies[comp].mean()
    # the term in parenthesis are force groups
    print(f'Mean energy per particle for {comp}: {mean_energy:.5f}')
    plt.plot(energies['Step'], energies[comp], label=f'{comp}')
    
plt.ylabel("Energy")
plt.xlabel("Simulation Step")
plt.legend(title="components")

In [ ]:
# Step 6b: Generate HiC map
hicman = HiCManager()
#Generating HiC from traj
hic = hicman.gen_hic_from_cndb(traj_file=traj_file, mu=2.0, rc=2.0, p=4.0, parallel=True)
plt.matshow(hic, cmap='Reds', norm=LogNorm())
plt.colorbar(shrink=0.7)